In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:

# pseduo code
import sys
sys.path.append("/projects/emty9224/GitCode/SNOCONE/")
from CNN_memoryOptimization import*
from CNN_preProcessing import*
from CNN_benchmarks import*
from CNN_modelArchitectureBlocks import* 
import rasterio
import shap
import pandas as pd
from rasterio.mask import mask
from tensorflow.keras.callbacks import ReduceLROnPlateau
from rasterio.windows import from_bounds
import psutil
from rasterio.transform import from_bounds 
import numpy as np
import sys
import os
import subprocess
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.utils import register_keras_serializable
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import to_categorical
from keras.models import Sequential, Model
from tensorflow.keras.models import load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, Dense, BatchNormalization, Activation, Input, Add
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import time
import matplotlib.pyplot as plt
from rasterio.transform import from_bounds
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import Sequence
from tensorflow.keras.utils import register_keras_serializable
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError
from tensorflow.keras.losses import Loss
import gc
import tensorflow.keras.backend as K

start_time = time.time()
print("modules established")

## establish file paths
start_year = 2022
end_year = 2024
Domain = "Rockies"
GPU = "N"
output_txt = "Y"
nonFreezeLayers = -8
penalty_weight = 1
penalties_used = ["fSCA", "LearningRateScaling"]
low_snow_weight=0   
swe_threshold=0.01
fsca_threshold=0.1
low_threshold=0.05
penalty_scale= 0
learningRateTesting = 1e-3

# workspaces
WorkspaceBase = f"/pl/active/hydroServer/ASO_DeepLearningData/CNN_v.02/{Domain}/"
phv_features = WorkspaceBase + "features/scaled/"
tree_workspace = WorkspaceBase + "treeCover/"
land_workspace = WorkspaceBase + "landCover/"
modelOuptuts = WorkspaceBase + "modelOutputs/"
DMFSCAWorkspace = WorkspaceBase + "Rockies_DMFSCA/"
final_activation = 'relu'
architectures = ['Baseline']  # Options are: Baseline, AdvancedBaseline, ResNet18, ResNet34, ResNet50, CustomSWE

test_groups = [
            ('Group1', 2025, 'G1'),
            ('Group2', 2025, 'G2'), 
            ('Group3', 2025, 'G3'),
            ('Group4', 2025, 'G4'),
            ('Group5', 2025, 'G5'),
            ('Group6', 2025, 'G6')
        ]

_mae_metric = MeanAbsoluteError()
_mse_metric = MeanSquaredError()
_rmse_metric = MeanSquaredError()

# # Check GPU availability
if GPU == "Y":
    print("\n Checking and start running on GPU")
    print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
    print("GPU devices:", tf.config.list_physical_devices('GPU'))
    
    # Configure GPU memory growth (prevents TensorFlow from allocating all GPU memory)
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            print("GPU memory growth enabled")
        except RuntimeError as e:
            print(e)
    
    # Verify TensorFlow is using GPU
    with tf.device('/GPU:0'):
        a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
        b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
        c = tf.matmul(a, b)
        print("GPU computation result:", c)

for architecture in architectures:
    print(architecture)
    # shapeChecks = "N"
    
    ## seting folder
    from datetime import datetime
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
   
    # create folder for model outputs
    os.makedirs(modelOuptuts + f"{str(timestamp)}/", exist_ok=True)
    inter_model_outWorkspace = modelOuptuts + f"{str(timestamp)}/"

    if output_txt == 'Y':
        f = open(inter_model_outWorkspace + f"code_output_{timestamp}.txt", "a")
        sys.stdout = f
        
    print("MODEL OUTPUTS TO BE PRINTED TO THIS DOC")
    print(f"{Domain}")
    print(f"\nPenalities: {penalties_used}")
    print(f"fSCA Penality Weight: {penalty_weight}")
    print(f"Low Snow Weight: {low_snow_weight}")
    print(f"Low Snow Threshold: {low_threshold}")
    print(f"Low Snow Penality Scale: {penalty_scale}")
    print("\n THIS CODE DOES DYNAMIC FREEZING LAYERS AND LEARNING RATES FOR TEST GROUPS")
    
    X, y, featureNames = target_feature_stacks(start_year=start_year, 
                                               end_year=end_year, 
                                               WorkspaceBase=WorkspaceBase, 
                                               ext = "nonull_fnl.tif", 
                                               vegetation_path = tree_workspace, 
                                               landCover_path = land_workspace, 
                                               phv_path = phv_features , 
                                               target_shape=(256,256),
                                               shapeChecks="Y")
    
    print("")
    print("Shape of input data")
    print(f"feature shape: {X.shape}")
    print(f"target shape: {y.shape}")
    feat_shape = X.shape
    featNo = feat_shape[-1]
    print(featNo)
    print(featureNames)
    
    # split between training and test data
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.15, shuffle=True)
    print("***")
    print("________________________________ Training and Validation Data Shapes ________________________________")
    print("Training data shape:", X_train.shape, y_train.shape)
    print("Validation data shape:", X_valid.shape, y_valid.shape)
    print("***")
    x_trainShape = X_train.shape
    x_validShape = X_valid.shape

    # Assuming featNo and final_activation are defined in your original code
    featNo = featNo  # Replace with your actual feature count
    final_activation = final_activation  # Replace with your actual activation
    
    # Create the model
    model = model_implementation(featNo, architecture, final_activation)
    
    # Your existing custom loss function
    from tensorflow.keras.losses import MeanSquaredError
    from tensorflow.keras.metrics import MeanAbsoluteError
    
    custom_loss_fn = make_swe_fsca_loss(
        base_loss_fn=MeanSquaredError(),
        penalty_weight=penalty_weight,
        swe_threshold=swe_threshold,
        fsca_threshold=fsca_threshold,
        mask_value=-1
    )
    
    # Compile with your existing setup
    model.compile(
        optimizer='adam',
        loss=custom_loss_fn,
        metrics=[masked_rmse, masked_mae, masked_mse]
    )
    
    print(model.summary())
    
    # establish the model
    best_weights_path = f"{inter_model_outWorkspace}/best_weights_{timestamp}.h5"
    checkpoint = ModelCheckpoint(best_weights_path, 
        monitor="val_loss",
        verbose=1, 
        save_best_only=True, 
        mode='min'
    )
    early_stopping = EarlyStopping(monitor="val_masked_rmse", mode='min', verbose=1, patience=10, restore_best_weights=True)
    
    
    batch_size = 32
    train_generator = DataGenerator(X_train, y_train, batch_size=batch_size)
    valid_generator = DataGenerator(X_valid, y_valid, batch_size=batch_size)
    
    history = model.fit(
        train_generator,
        validation_data=valid_generator,
        epochs=100,
        callbacks=[checkpoint, early_stopping],
        verbose=0
    )

    # manual backup save of weights
    backup_weights_path = f"{inter_model_outWorkspace}/backup_weights_{timestamp}.h5"
    try:
        model.save_weights(backup_weights_path)
        print(f"Backup weights saved to: {backup_weights_path}")
    except Exception as e:
        print(f"Failed to save backup weights: {e}")
    
    # Also save model architecture as JSON for SHAP compatibility
    model_json = model.to_json()
    with open(f"{inter_model_outWorkspace}/model_architecture_{timestamp}.json", "w") as json_file:
        json_file.write(model_json)
    print("Model architecture saved as JSON")

    ## plotting results 
    fig, axs = plt.subplots(3, 1, figsize=(12, 10), sharex=True)
    # 1. Masked MSE (used as loss)
    axs[0].plot(history.history['loss'], label='Train MSE Loss')
    axs[0].plot(history.history['val_loss'], label='Val MSE Loss')
    axs[0].set_ylabel('MSE Loss')
    axs[0].set_title('Masked MSE')
    axs[0].legend()
    axs[0].grid(True)
    
    # 2. Masked RMSE
    axs[1].plot(history.history['masked_rmse'], label='Train RMSE')
    axs[1].plot(history.history['val_masked_rmse'], label='Val RMSE')
    axs[1].set_ylabel('RMSE')
    axs[1].set_title('Masked RMSE')
    axs[1].legend()
    axs[1].grid(True)
    
    # 3. Masked MAE
    axs[2].plot(history.history['masked_mae'], label='Train MAE')
    axs[2].plot(history.history['val_masked_mae'], label='Val MAE')
    axs[2].set_ylabel('MAE')
    axs[2].set_title('Masked MAE')
    axs[2].set_xlabel('Epoch')
    axs[2].legend()
    axs[2].grid(True)
    
    plt.tight_layout()
    plt.savefig(inter_model_outWorkspace + "Model_error_epochs.png")
    plt.show()
    
    ## Add metrics to recurring error tracking sheet
    metrics_to_track = ['val_masked_rmse', 'val_masked_mse', 'val_masked_mae']
    best_metrics = {}
    print("\nValidation Metric Progression:")
    for metric in metrics_to_track:
        values = history.history.get(metric, [])
        if values:
            best_val = min(values)
            best_metrics[metric] = best_val
            print(f"{metric}: Start = {values[0]:.4f}, End = {values[-1]:.4f}")
            print(f"{metric}: Best = {best_val:.4f}")
        else:
            print(f"{metric}: Not found in history.")
    print(f"Final activation function: {final_activation}")
    
    # Example variables
    feature_csv = modelOuptuts + f"{Domain}_model_featureList_summary.csv" 
    column_name = timestamp  
    feature_list = featureNames    
    new_column_df = pd.DataFrame({column_name: feature_list})
    
    # If the file already exists, load it and append the new column
    if os.path.exists(feature_csv):
        existing_df = pd.read_csv(feature_csv)
        # Reindex the existing dataframe 
        max_len = max(len(existing_df), len(new_column_df))
        existing_df = existing_df.reindex(range(max_len))
        new_column_df = new_column_df.reindex(range(max_len))
        # Combine horizontally
        combined_df = pd.concat([existing_df, new_column_df], axis=1)
    else:
        combined_df = new_column_df
    
    # Save back to CSV
    combined_df.to_csv(feature_csv, index=False)
    
    # add metrics to csv
    modelStatsCSV = modelOuptuts + f"{Domain}_modelSummary_stats.csv"
    
    baseline_stats = {
        'ModelRun':[timestamp],
        'FeatureNum': [featNo],
        'Architecture': [architecture],
        'FinalActivation': [final_activation],
        'X_TrainShape': [x_trainShape[0]],
        'X_ValidShape': [x_validShape[0]],
        'RMSE': [best_metrics['val_masked_rmse']],
        'MSE': [best_metrics['val_masked_mse']], 
        'MAE': [best_metrics['val_masked_mae']]
        
    }
    
    df = pd.DataFrame(baseline_stats)
    
    # Append or write new file
    if os.path.exists(modelStatsCSV):
        df.to_csv(modelStatsCSV, mode='a', header=False, index=False)
    else:
        df.to_csv(modelStatsCSV, index=False)
    
    del X_train, X_valid, y_train, y_valid, X, y
    K.clear_session()
    gc.collect()

    del history 
    clear_memory()

    # create folder
    print("\n ############### MOVING TO TEST GROUPS OF DATA ###############")
    for group_name, year, group_id in test_groups:
        print(f"\nTesting {group_name}")
        print(f"Adjusting the last {nonFreezeLayers} layers")
        print(f"Learning rate = {learningRateTesting}")
        print(f"fSCA Penalty rate = {penalty_weight}")
        
        # Create output folder
        out_folder = inter_model_outWorkspace + f"outTifs_{group_id}_yPreds_tifs/"
        os.makedirs(out_folder, exist_ok=True)
        print("made output directory")
        
        # Load test data
        X_train_g, y_train_g, g_train_extents, g_train_crs = target_feature_stacks_testGroups(
            year=year,
            target_splits_path=WorkspaceBase + f"test_groups/{group_name}/train/",
            fSCA_path=WorkspaceBase + f"{year}/fSCA/",
            DMFSCA_path=WorkspaceBase + f"{year}/DMFSCA/",
            vegetation_path=WorkspaceBase + "treeCover/",
            landCover_path=land_workspace,
            phv_path=WorkspaceBase + "features/scaled/",
            extension_filter=".tif",
            desired_shape=(256, 256),
            debug_output_folder="./debug_outputs/",
            num_of_channels=featNo,
            shapeChecks="Y"
        )
        
        X_test_g, y_test_g, g_test_extents, g_test_crs = target_feature_stacks_testGroups(
            year=year,
            target_splits_path=WorkspaceBase + f"test_groups/{group_name}/test/",
            fSCA_path=WorkspaceBase + f"{year}/fSCA/",
            DMFSCA_path=WorkspaceBase + f"{year}/DMFSCA/",
            vegetation_path=WorkspaceBase + "treeCover/",
            landCover_path=land_workspace,
            phv_path=WorkspaceBase + "features/scaled/",
            extension_filter=".tif",
            desired_shape=(256, 256),
            debug_output_folder="./debug_outputs/",
            num_of_channels=featNo,
            shapeChecks="Y"
        )
        
        # Convert to arrays
        X_train_array = np.array(X_train_g)
        y_train_array = np.array(y_train_g)
        X_test_array = np.array(X_test_g)
        y_test_array = np.array(y_test_g)
        print(f"X_train shape: {X_train_array.shape}")
        print(f"y_train shape: {y_train_array.shape}")
        print(f"X_test shape: {X_test_array.shape}")
        print(f"y_test shape: {y_test_array.shape}")

        # reload loss functions
        custom_loss_fn = make_swe_fsca_loss(
            base_loss_fn=MeanSquaredError(),
            penalty_weight=penalty_weight,
            swe_threshold=swe_threshold,
            fsca_threshold=fsca_threshold,
            mask_value=-1
        )
        # add in test model with the weights and test different weights options
        operational_model = model_implementation(featNo, architecture, final_activation)

        # Try multiple weight file locations
        weight_files = [
            f"{inter_model_outWorkspace}/best_weights_{timestamp}.h5",
            f"{inter_model_outWorkspace}/backup_weights_{timestamp}.h5",
            f"{inter_model_outWorkspace}/best_weights_{timestamp}.weights.h5"
        ]
        
        weights_loaded = False
        for weight_file in weight_files:
            if os.path.exists(weight_file):
                try:
                    print(f"Attempting to load weights from: {weight_file}")
                    operational_model.load_weights(weight_file)
                    print(f"Successfully loaded weights from: {weight_file}")
                    weights_loaded = True
                    break
                except Exception as e:
                    print(f"Failed to load weights from {weight_file}: {e}")

        if not weights_loaded:
            print("ERROR: Could not load any weights file. Check if training completed successfully.")
            print("Available files in directory:")
            for file in os.listdir(inter_model_outWorkspace):
                if 'weights' in file or file.endswith('.h5'):
                    print(f"  - {file}")
            continue  # Skip this test group

        # Test different freezing levels and train with best one
        print(f"\n Testing and training with optimal freezing for {group_name}...")
        test_levels = [-4, -6, -8, -10, -12]
        print(f"Testing levels: {test_levels}")
        
        freezing_results = {}
        best_freezing_level = -8  
        best_val_loss = float('inf')
        
        for freeze_level in test_levels:
            print(f"\n--- Testing {freeze_level} unfrozen layers ---")
            
            # Reset all layers to trainable first
            for layer in operational_model.layers:
                layer.trainable = True
            
            # Apply current freezing level
            for layer in operational_model.layers[:freeze_level]:
                layer.trainable = False
            
            frozen_count = len([l for l in operational_model.layers if not l.trainable])
            trainable_count = len([l for l in operational_model.layers if l.trainable])
            
            print(f"Frozen: {frozen_count}, Trainable: {trainable_count}")
            
            # Learning rate logger for this test
            class LearningRateLogger(tf.keras.callbacks.Callback):
                def __init__(self):
                    super().__init__()
                    self.lrs = []
                    
                def on_epoch_end(self, epoch, logs=None):
                    lr = self.model.optimizer.learning_rate.numpy()
                    self.lrs.append(lr)
            
            lr_logger = LearningRateLogger()
            
            # Compile with this configuration
            operational_model.compile(
                optimizer=Adam(learning_rate=learningRateTesting),
                loss=custom_loss_fn,
                metrics=[masked_rmse, masked_mae, masked_mse]
            )
            
            # Setup callbacks for this test
            test_early_stop = EarlyStopping(
                monitor='val_loss', 
                patience=5,  # Shorter patience for testing
                restore_best_weights=True,
                verbose=0
            )
            
            test_reduce_lr = ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=2,  # Shorter patience for testing
                min_lr=1e-6,
                verbose=0
            )
            
            # Train with full fine-tuning setup (not just 5 epochs)
            test_history = operational_model.fit(
                X_train_array, y_train_array,
                batch_size=16,
                epochs=15,  # Reasonable epochs for testing
                validation_split=0.2,
                callbacks=[test_early_stop, test_reduce_lr, lr_logger],
                verbose=0  # Silent for testing
            )
            
            # Get final validation loss
            final_val_loss = test_history.history['val_loss'][-1]
            final_val_rmse = test_history.history['val_masked_rmse'][-1]
            best_val_loss_in_test = min(test_history.history['val_loss'])
            
            # Check for improvement trend
            val_losses = test_history.history['val_loss']
            improvement = val_losses[0] - val_losses[-1]
            
            # Learning rate info
            final_lr = lr_logger.lrs[-1]
            lr_reductions = sum(1 for i in range(1, len(lr_logger.lrs)) if lr_logger.lrs[i] < lr_logger.lrs[i-1])
            
            freezing_results[freeze_level] = {
                'final_val_loss': final_val_loss,
                'best_val_loss': best_val_loss_in_test,
                'final_val_rmse': final_val_rmse,
                'improvement': improvement,
                'frozen_count': frozen_count,
                'trainable_count': trainable_count,
                'final_lr': final_lr,
                'lr_reductions': lr_reductions,
                'epochs_trained': len(test_history.history['loss']),
                'lr_progression': lr_logger.lrs
            }
            
            print(f"Val Loss: {final_val_loss:.4f}, Best: {best_val_loss_in_test:.4f}, Final LR: {final_lr:.2e}, LR Reductions: {lr_reductions}")
            
            # Track the overall best
            if best_val_loss_in_test < best_val_loss:
                best_val_loss = best_val_loss_in_test
                best_freezing_level = freeze_level
        
        print(f"\n Best freezing level for {group_name}: {best_freezing_level} (Best Val Loss: {best_val_loss:.4f})")
        
        # Apply the best freezing level for final results
        for layer in operational_model.layers[:best_freezing_level]:
            layer.trainable = False
        
        frozen_count = len([l for l in operational_model.layers if not l.trainable])
        trainable_count = len([l for l in operational_model.layers if l.trainable])
        
        print(f"Applied optimal freezing: {frozen_count} frozen, {trainable_count} trainable layers")
        
        # Get the best results for reporting
        best_results = freezing_results[best_freezing_level]
        final_lr = best_results['final_lr']
        lr_reductions = best_results['lr_reductions']
        adaptation_epochs = best_results['epochs_trained']
        
        print(f"Best configuration achieved:")
        print(f"  Final LR: {final_lr:.2e}")
        print(f"  LR reductions: {lr_reductions}")
        print(f"  Epochs trained: {adaptation_epochs}")
        

        # After fine-tuning, analyze the adaptation process
        print(f"\n{group_name} Adaptation Analysis:")
        print(f"Best val_loss achieved: {best_results['best_val_loss']:.4f}")
        print(f"Final val_loss: {best_results['final_val_loss']:.4f}")
        print(f"Total epochs trained: {adaptation_epochs}")
        
        # Calculate overfitting gap from best results
        overfitting_gap = best_results['final_val_loss'] - best_results['best_val_loss']
        print(f"Overfitting gap: {overfitting_gap:.4f}")
        if overfitting_gap > 0.1:
            print("Potential overfitting detected!")
        
        # Enhanced learning rate analysis
        print(f"\nLearning Rate Summary:")
        print(f"Initial LR: {learningRateTesting:.2e}")
        print(f"Final LR: {final_lr:.2e}")
        print(f"LR reductions: {lr_reductions}")
        
        # Show LR progression if you want detail
        if len(best_results['lr_progression']) > 1:
            print(f"LR progression: {best_results['lr_progression'][0]:.2e} → {final_lr:.2e}")
        # Make predictions
        print("Making predictions on test data...")
        predictions = operational_model.predict(X_test_array, batch_size=32, verbose=1)

        # Evaluate the model on test data
        print("Evaluating model on test data...")
        test_metrics = operational_model.evaluate(X_test_array, y_test_array, batch_size=32, verbose=1)

        # Print test metrics
        print(f"\n{group_name} Operational Results (Adapted to 2025):")
        print(f"Masked MSE (Loss): {test_metrics[0]:.4f}")
        print(f"Masked RMSE:       {test_metrics[1]:.4f}")
        print(f"Masked MAE:        {test_metrics[2]:.4f}")
        print(f"Masked MSE Metric: {test_metrics[3]:.4f}")

        print("\n--- Baseline Comparison (No Adaptation) ---")
        baseline_model = model_implementation(featNo, architecture, final_activation)
        baseline_model.load_weights(best_weights_path)
        baseline_model.compile(
            optimizer='adam',
            loss=custom_loss_fn,
            metrics=[masked_rmse, masked_mae, masked_mse]
        )
        
        # Process predictions and save
        for i, pred in enumerate(predictions): # CHECK TO SEE IF THIS IS THE CORRECT SYNTAX
            array = pred.reshape((256, 256))
            mask = y_test_g[i].reshape((256, 256)) != -1
            array_masked = np.where(mask, array, -1)
            
            save_array_as_raster(
                output_path=f"{out_folder}/prediction_{i}.tif",
                array=array_masked.astype(np.float32),
                extent=g_test_extents[i],
                crs=g_test_crs[i],
                nodata_val=-1
            )
            
            del array, mask, array_masked
            
        adaptation_metrics = {
            'Group': [group_name],
            'Baseline_RMSE': [baseline_stats['RMSE'][0]],
            'Adapted_RMSE': [test_metrics[1]],
            'Adapted_MSE': [test_metrics[3]],
            'Adapted_MAE': [test_metrics[2]],
            'Improvement': [baseline_stats['RMSE'][0] - test_metrics[1]],
            'Adaptation_Epochs': [adaptation_epochs],
            'Final_Val_Loss': [best_results['best_val_loss']],
            'Initial_LR': [learningRateTesting],
            'Final_LR': [final_lr],
            'LR_Reductions': [lr_reductions],
            'Overfitting_Gap': [overfitting_gap],
            'Optimal_Freeze_Level': [best_freezing_level],
            'Frozen_Layers': [frozen_count],
            'Trainable_Layers': [trainable_count]   
        }
                
        adaptation_df = pd.DataFrame(adaptation_metrics)
        adaptation_csv = inter_model_outWorkspace + f"operational_adaptation_results.csv"
        
        if os.path.exists(adaptation_csv):
            adaptation_df.to_csv(adaptation_csv, mode='a', header=False, index=False)
        else:
            adaptation_df.to_csv(adaptation_csv, index=False)
            
        # Cleanup
        del operational_model, baseline_model, custom_loss_fn
        del X_train_g, y_train_g, g_train_extents, g_train_crs
        del X_test_g, y_test_g, g_test_extents, g_test_crs
        del X_train_array, y_train_array, X_test_array, y_test_array
        del predictions, test_metrics
        # del adaptation_history  # Remove this line
        
        clear_memory()
        print(f"{group_name} operational testing completed\n")
        
        # End timing for this architecture
        end_time = time.time()
        total_minutes = (end_time - start_time) / 60
        print(f"\nArchitecture {architecture} completed in {total_minutes:.2f} minutes")

    try:
        f.close()
        sys.stdout = sys.__stdout__  # Reset stdout
    except:
        pass
    clear_memory()


